<a href="https://colab.research.google.com/github/vitorcodesalittle/dataset-supermercado/blob/master/ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

print(tf.__version__)

2.5.0


In [ ]:
!wget https://raw.githubusercontent.com/vitorcodesalittle/dataset-supermercado/master/information-extraction/labels.txt
!wget https://raw.githubusercontent.com/vitorcodesalittle/dataset-supermercado/master/information-extraction/sentences.txt

--2021-08-03 21:14:12--  https://raw.githubusercontent.com/vitorcodesalittle/dataset-supermercado/master/information-extraction/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127 [text/plain]
Saving to: ‘labels.txt.1’

labels.txt.1        100%[===================>]     127  --.-KB/s    in 0s      

2021-08-03 21:14:14 (4.82 MB/s) - ‘labels.txt.1’ saved [127/127]

--2021-08-03 21:14:14--  https://raw.githubusercontent.com/vitorcodesalittle/dataset-supermercado/master/information-extraction/sentences.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awai

In [ ]:
def file2Examples(labels, sentences):  
  examples=[]

  with open(labels,"r") as l:
    with open(sentences, "r") as s:
      for label in l:
        example = [[],[]]
        sentence = s.readline()

        label_split= label.split()
        sentence_split = sentence.split()
          
        if len(label_split) == len(sentence_split):
          example[0] = sentence_split
          example[1] = label_split
          examples.append(example)
    
    return examples

examples = file2Examples("labels.txt", "sentences.txt")
print(examples)

[[['Coloca', 'uma', 'manteiga'], ['O', 'BQ', 'BP']], [['Tira', 'a', 'manteiga'], ['O', 'O', 'BP']], [['Adicionar', 'açucar', 'demerara', '500g'], ['O', 'BP', 'IP', 'BQ']], [['Inserir', 'peito', 'de', 'frango', 'fatiado'], ['O', 'BP', 'IP', 'IP', 'IP']], [['Colocar', 'cerveja', 'colorado', 'appia'], ['O', 'BP', 'BM', 'IP']], [['Tira', 'a', 'manteiga'], ['O', 'O', 'BP']], [['Remover', 'presunto', 'sadia'], ['O', 'BP', 'BM']], [['Tirar', 'palma', 'banana', 'prata'], ['O', 'BQ', 'BP', 'IP']], [['Excluir', 'queijo', 'do', 'reino'], ['O', 'BP', 'IP', 'IP']], [['Retirar', 'chocolate', 'diamante', 'negro'], ['O', 'BP', 'BM', 'IM']], [['Ver', 'pão', 'francês'], ['O', 'BP', 'IP']], [['Mostrar', 'detergente'], ['O', 'BP']], [['Vinho', 'branco', 'albaclara', 'sauvignon', 'blanc'], ['BP', 'IP', 'BM', 'IM', 'IM']]]


In [ ]:
    # create character vocab
    labels = ['O', 'BP', 'BM', 'IM', 'IP', 'BQ']
    all_text = " ".join([" ".join(x[0]) for x in examples])
    vocab = sorted(set(all_text))
    
    # create character/id and label/id mapping
    char2idx = {u:i+1 for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    label2idx = {u:i+1 for i, u in enumerate(labels)}
    idx2label = np.array(labels)
    
    print(idx2label)
    print(char2idx)

['O' 'BP' 'BM' 'IM' 'IP' 'BQ']
{' ': 1, '0': 2, '5': 3, 'A': 4, 'C': 5, 'E': 6, 'I': 7, 'M': 8, 'R': 9, 'T': 10, 'V': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'x': 33, 'ã': 34, 'ç': 35, 'ê': 36}


In [ ]:
def split_char_labels(eg):
      
      tokens = eg[0]
      labels= eg[1]

      input_chars = []
      output_char_labels = []

      for token,label in zip(tokens,labels):

        input_chars.extend([char for char in token])
        input_chars.extend(' ')
        output_char_labels.extend([label]*len(token))
        output_char_labels.extend('O')

      return [[char2idx[x] for x in input_chars[:-1]],np.array([label2idx[x] for x in output_char_labels[:-1]])]
   
train_formatted = [split_char_labels(eg) for eg in examples]

print(len(train_formatted))


[[[5, 25, 22, 25, 14, 12, 1, 31, 23, 12, 1, 23, 12, 24, 30, 16, 20, 18, 12], array([1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 1, 2, 2, 2, 2, 2, 2, 2, 2])], [[10, 20, 28, 12, 1, 12, 1, 23, 12, 24, 30, 16, 20, 18, 12], array([1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2])], [[4, 15, 20, 14, 20, 25, 24, 12, 28, 1, 12, 35, 31, 14, 12, 28, 1, 15, 16, 23, 16, 28, 12, 28, 12, 1, 3, 2, 2, 18], array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 5, 5, 5, 5, 5,
       5, 5, 5, 1, 6, 6, 6, 6])], [[7, 24, 29, 16, 28, 20, 28, 1, 26, 16, 20, 30, 25, 1, 15, 16, 1, 17, 28, 12, 24, 18, 25, 1, 17, 12, 30, 20, 12, 15, 25], array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 5, 5, 1, 5, 5, 5, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 5])], [[5, 25, 22, 25, 14, 12, 28, 1, 14, 16, 28, 32, 16, 21, 12, 1, 14, 25, 22, 25, 28, 12, 15, 25, 1, 12, 26, 26, 20, 12], array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 5, 5, 5, 5, 5])], [[10, 20, 28, 12, 1, 12, 1, 23, 12, 24, 30, 16, 20, 18, 12]

In [ ]:
    # training generator
    def gen_train_series():

        for eg in train_formatted:
          yield eg[0],eg[1]
      
    # create Dataset objects for train, test and validation sets  
    series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

    BATCH_SIZE = 128
    BUFFER_SIZE = 1000
    
    # create padded batch series objects for train, test and validation sets
    ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
    
    # print example batches
    for input_example_batch, target_example_batch in ds_series_batch:
      print(input_example_batch)
      print(target_example_batch)

In [ ]:
  vocab_size = len(vocab)+1

  # The embedding dimension
  embedding_dim = 256

  # Number of RNN units
  rnn_units = 1024

  label_size = len(labels)  
  
  # build LSTM model
  def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None],mask_zero=True),
            tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(label_size)
            ])
        return model

  model = build_model(
        vocab_size = len(vocab)+1,
        label_size=len(labels)+1,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units,
        batch_size=BATCH_SIZE)

  model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          9472      
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
dense (Dense)                (128, None, 7)            7175      
Total params: 5,263,623
Trainable params: 5,263,623
Non-trainable params: 0
_________________________________________________________________


In [ ]:
    import os

    # define loss function
    def loss(labels, logits):
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

    model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    # Directory where the checkpoints will be saved
    checkpoint_dir = './training_checkpoints'
    # Name of the checkpoint files
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

In [ ]:
EPOCHS=20
  
history = model.fit(ds_series_batch, epochs=EPOCHS,callbacks=[checkpoint_callback])

Epoch 1/20


ValueError: ignored